<a href="https://colab.research.google.com/github/quang-m-nguyen/DeepPGD/blob/main/dna_bert2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

!pip install einops

from transformers.models.bert.configuration_bert import BertConfig
import torch
from transformers import AutoTokenizer, AutoModel

config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

In [ ]:
def get_embedding(dna):
  inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
  hidden_states = model(inputs)[0] # [1, sequence_length, 768]
  embedding_mean = torch.mean(hidden_states[0], dim=0)
  return embedding_mean

dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
embedding = get_embedding(dna)
print(len(embedding))

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv', delimiter='\t')
df_limited = df.head(5)
display(df_limited)

In [ ]:
import pickle
import os
import pandas as pd

def process_tsv_and_save_embeddings(tsv_file_path):
    # Read the TSV file
    df = pd.read_csv(tsv_file_path, delimiter='\t')
    embeddings_data = []
    count = 0

    # Process each DNA sequence
    for index, row in df.iterrows():
        dna = row['text']
        embedding = get_embedding(dna)
        embeddings_data.append({'dna': dna, 'embedding': embedding})
        count += 1
        print(f"Finished processing DNA sequence {count}: {dna}, embedding shape: {embedding.shape}")

    # Determine the output file path
    tsv_dir = os.path.dirname(tsv_file_path)
    tsv_filename = os.path.basename(tsv_file_path).replace('.tsv', '')
    output_file_path = os.path.join(tsv_dir, f"{tsv_filename}_embeddings_data.pkl")

    # Save the embeddings data as a pickle file
    with open(output_file_path, 'wb') as f:
        pickle.dump(embeddings_data, f)

    print(f"Embeddings data saved to {output_file_path}")

# Example usage
# tsv_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv'
# process_tsv_and_save_embeddings(tsv_file_path)

In [ ]:
# List of TSV file paths to process
tsv_file_paths = [
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train.tsv'



    # '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/test.tsv',
    # '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/train.tsv',
    # '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv'
]

# Loop through each file path and process it
for tsv_file_path in tsv_file_paths:
    process_tsv_and_save_embeddings(tsv_file_path)

In [ ]:
import pickle

def read_embeddings_data(pkl_file_path, num_rows=1):
    with open(pkl_file_path, 'rb') as f:
        embeddings_data = pickle.load(f)

    limited_data = embeddings_data[:num_rows]
    for item in limited_data:
        print(f"DNA: {item['dna']}, Embedding shape: {item['embedding'].shape}")

# Example usage
pkl_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train_embeddings_data.pkl'
read_embeddings_data(pkl_file_path)